IMPORTANT : Définition des positions !
(x,y)

x : Positif -> Devant / Négatif -> Derrière

y : Positif -> Gauche / Négatif -> Droite 

In [1]:
import math,random

In [2]:
def webfrac(x=1,nb=5, k = 0.9):
    """
    Trouve une valeur Z tel que la weberFraction(x,y)=weberFraction(y,z)
    Valeur de référence : a distance de 15, il faut au moins une seconde distance de 18 pour differencier
    """
    
    
    res = [x]
    for i in range(nb-1):
        res.append(round(res[-1]*k+res[-1]))
    return res

In [3]:
dists = webfrac(5,5,k=0.7)
dists

[5, 8, 14, 24, 41]

In [4]:
def convertDistanceAngleEnPosition(angle,distance, ecartDistance = 5, logs = False, dist = []):
    if(logs):
        r = dist[distance]
    else:
        r = distance*ecartDistance
    #Shift pour lisibilité : Pour la simulation 0° est devant 90° est a gauche
    angle = angle-90
    posX = r*math.cos(math.radians(angle))
    posY = r*math.sin(math.radians(angle))
    return (round(posX),round(posY))


In [5]:
def convertPositionEnAngleDistance(x,y):
    d = math.sqrt(x**2+y**2)
    segO = [(0,0),(50,0)]
    Xa,Ya = segO[1]
    seg1 = [(0,0),(x,y)]
    Xb,Yb = seg1[1]
    cosA = (Xa*Xb+Ya*Yb) / math.sqrt((Xa**2+Ya**2)*(Xb**2+Yb**2))
    a=math.degrees(math.acos(cosA))
    d = x/math.cos(math.radians(a))
    d2 = y/math.sin(math.radians(a))
    return round(a),round(d),round(d2)

In [6]:
convertPositionEnAngleDistance(20,36)

(61, 41, 41)

In [7]:
def calculPositionDisponibles(nombreAngle = 7,nombreDistance=4, espaceDistance = 5, logs = False, dist = dists):
    """Détermine l'ensemble des positions disponibles en fonctions du nombre d'angles et de cercles de positionnement accessible"""
    position = []
    deltaAngle = 180 / (nombreAngle - 1)
    if(logs):
        for d in range(0,len(dist)):
            for a in range(0, nombreAngle):
                angle = a * deltaAngle
                x,y = convertDistanceAngleEnPosition(angle,d,logs=logs,dist=dist)
                position.append((round(x),round(y)))
    else:
        for d in range(1,nombreDistance+1):
            for a in range(0, nombreAngle):
                angle = a * deltaAngle
                x,y = convertDistanceAngleEnPosition(angle,d)
                position.append((round(x),round(y)))
    return position

In [8]:
#Decrit les lignes de code a mettre dans le code c++ pour enregister les positions de sons de moustiques
a= calculPositionDisponibles(logs=True,dist=dists)
s=""
for i,j in a:
    s += "pathSave.push_back({ "+str(i)+","+str(j)+" });"+"\n"
print(s)

pathSave.push_back({ 0,-5 });
pathSave.push_back({ 3,-4 });
pathSave.push_back({ 4,-2 });
pathSave.push_back({ 5,0 });
pathSave.push_back({ 4,2 });
pathSave.push_back({ 3,4 });
pathSave.push_back({ 0,5 });
pathSave.push_back({ 0,-8 });
pathSave.push_back({ 4,-7 });
pathSave.push_back({ 7,-4 });
pathSave.push_back({ 8,0 });
pathSave.push_back({ 7,4 });
pathSave.push_back({ 4,7 });
pathSave.push_back({ 0,8 });
pathSave.push_back({ 0,-14 });
pathSave.push_back({ 7,-12 });
pathSave.push_back({ 12,-7 });
pathSave.push_back({ 14,0 });
pathSave.push_back({ 12,7 });
pathSave.push_back({ 7,12 });
pathSave.push_back({ 0,14 });
pathSave.push_back({ 0,-24 });
pathSave.push_back({ 12,-21 });
pathSave.push_back({ 21,-12 });
pathSave.push_back({ 24,0 });
pathSave.push_back({ 21,12 });
pathSave.push_back({ 12,21 });
pathSave.push_back({ 0,24 });
pathSave.push_back({ 0,-41 });
pathSave.push_back({ 21,-36 });
pathSave.push_back({ 36,-20 });
pathSave.push_back({ 41,0 });
pathSave.push_back({ 36,20 });
pa

In [9]:
def calculMouvements(nombreAngle = 7, nombreDistance = 4, espaceDistance = 5, logs = False, dist = dists,verbose= True):
    """
    a : approche
    e : eloignement
    g : mvm gauche
    d : mvm droite
    ag,ad,eg,ed
    """
    pos = calculPositionDisponibles(nombreAngle = 7, nombreDistance = 4, espaceDistance = 5, logs = logs, dist = dist)
    angle = round(180 / (nombreAngle-1))
    depl = dict()
    cptA = 0
    cptE = 0
    cptG = 0
    cptD = 0
    cptAG = 0
    cptAD = 0
    cptEG = 0
    cptED = 0
    #Uniquement jump d'une case et pas de deux
    cpt =0
    if(logs):
        for an in range(nombreAngle):
            for d in range(0,len(dist)):
                a = an*angle
                pos = convertDistanceAngleEnPosition(a,d,logs=True,dist=dist)
                if(verbose):
                    print("Position intiales :"+str(pos))
                    cpt+=1
                depl[pos] = dict()
                if(d != 0):
                    depl[pos]["a"] = convertDistanceAngleEnPosition(a,d-1,logs=True,dist=dist)
                    cptA+=1
                    if(a-angle >= 0):
                        depl[pos]["ad"] = convertDistanceAngleEnPosition(a-angle,d-1,logs=True,dist=dist)
                        cptAG+=1
                    if(a +angle <= 180):
                        depl[pos]["ag"] = convertDistanceAngleEnPosition(a+angle,d-1,logs=True,dist=dist)
                        cptAD+=1

                if(d >= 2):
                    depl[pos]["aa"] = convertDistanceAngleEnPosition(a,d-2,logs=True,dist=dist)
                    cptA+=1
                    if(a >= angle*2):
                        depl[pos]["aadd"] = convertDistanceAngleEnPosition(a-(angle*2),d-2,logs=True,dist=dist)
                        cptAG+=1
                    if(a +(angle*2) <= 180):
                        depl[pos]["aagg"] = convertDistanceAngleEnPosition(a+(angle*2),d-2,logs=True,dist=dist)
                        cptAD+=1

                if(d < len(dist)-1):
                    depl[pos]["e"] = convertDistanceAngleEnPosition(a,d+1,logs=True,dist=dist)
                    cptE+=1
                    if(a - angle >= 0):
                        depl[pos]["ed"] = convertDistanceAngleEnPosition(a-angle,d+1,logs=True,dist=dist)
                        cptEG+=1
                    if(a + angle <= 180):
                        depl[pos]["eg"] = convertDistanceAngleEnPosition(a+angle,d+1,logs=True,dist=dist)
                        cptED+=1
                if(d < len(dist)-2):
                    depl[pos]["ee"] = convertDistanceAngleEnPosition(a,d+2,logs=True,dist=dist)
                    cptE+=1
                    if(a >= (angle*2)):
                        depl[pos]["eedd"] = convertDistanceAngleEnPosition(a-(angle*2),d+2,logs=True,dist=dist)
                        cptEG+=1
                    if(a+(angle*2) <= 180):
                        depl[pos]["eegg"] = convertDistanceAngleEnPosition(a+(angle*2),d+2,logs=True,dist=dist)
                        cptED+=1
                if(a != 0):
                    depl[pos]["d"] = convertDistanceAngleEnPosition(a-angle,d,logs=True,dist=dist)
                    cptG+=1
                if(a >=(angle*2)):
                    depl[pos]["dd"] = convertDistanceAngleEnPosition(a-(angle*2),d,logs=True,dist=dist)
                    cptG+=1

                if(a +angle <= 180):
                    depl[pos]["g"] = convertDistanceAngleEnPosition(a+angle,d,logs=True,dist=dist)
                    cptG+=1
                if(a +angle*2 <= 180):
                    depl[pos]["gg"] = convertDistanceAngleEnPosition(a+(angle*2),d,logs=True,dist=dist)
                    cptD+=1
    else:
        for an in range(nombreAngle):
            for d in range(1,nombreDistance+1):
                a = an*angle
                pos = convertDistanceAngleEnPosition(a,d)
                if(verbose):
                    print("Position intiales :"+str(pos))
                    cpt+=1
                depl[pos] = dict()
                if(d != 1):
                    depl[pos]["a"] = convertDistanceAngleEnPosition(a,d-1)
                    cptA+=1
                    if(a-angle >= 0):
                        depl[pos]["ad"] = convertDistanceAngleEnPosition(a-angle,d-1)
                        cptAG+=1
                    if(a +angle <= 180):
                        depl[pos]["ag"] = convertDistanceAngleEnPosition(a+angle,d-1)
                        cptAD+=1

                if(d >= 2):
                    depl[pos]["aa"] = convertDistanceAngleEnPosition(a,d-2)
                    cptA+=1
                    if(a >= angle*2):
                        depl[pos]["aadd"] = convertDistanceAngleEnPosition(a-(angle*2),d-2)
                        cptAG+=1
                    if(a +(angle*2) <= 180):
                        depl[pos]["aagg"] = convertDistanceAngleEnPosition(a+(angle*2),d-2)
                        cptAD+=1

                if(d != nombreDistance):
                    depl[pos]["e"] = convertDistanceAngleEnPosition(a,d+1)
                    cptE+=1
                    if(a - angle >= 0):
                        depl[pos]["ed"] = convertDistanceAngleEnPosition(a-angle,d+1)
                        cptEG+=1
                    if(a + angle <= 180):
                        depl[pos]["eg"] = convertDistanceAngleEnPosition(a+angle,d+1)
                        cptED+=1
                if(d <= nombreDistance-1):
                    depl[pos]["ee"] = convertDistanceAngleEnPosition(a,d+2)
                    cptE+=1
                    if(a >= (angle*2)):
                        depl[pos]["eedd"] = convertDistanceAngleEnPosition(a-(angle*2),d+2)
                        cptEG+=1
                    if(a+(angle*2) <= 180):
                        depl[pos]["eegg"] = convertDistanceAngleEnPosition(a+(angle*2),d+2)
                        cptED+=1
                if(a != 0):
                    depl[pos]["d"] = convertDistanceAngleEnPosition(a-angle,d)
                    cptG+=1
                if(a >=(angle*2)):
                    depl[pos]["dd"] = convertDistanceAngleEnPosition(a-(angle*2),d)
                    cptG+=1

                if(a +angle <= 180):
                    depl[pos]["g"] = convertDistanceAngleEnPosition(a+angle,d)
                    cptG+=1
                if(a +angle*2 <= 180):
                    depl[pos]["gg"] = convertDistanceAngleEnPosition(a+(angle*2),d)
                    cptD+=1
    if(verbose): 
        print("Nombre de pos initiales étudiées "+str(cpt))
        print("Nombre de positions initiales : "+str(len(depl.keys())))
        print(depl)
        print("A"+str(cptA))
        print("AG"+str(cptAG))
        print("AD"+str(cptAD))
        print("E"+str(cptE))
        print("EG"+str(cptEG))
        print("ED"+str(cptED))
        print("G"+str(cptG))
        print("D"+str(cptD))
        print("Nombre mouvement sans diagonales")
        print(cptA+cptE+cptG+cptD)
        print("Nombre mouvement avec diagonales")
        print(cptA+cptAG+cptAD+cptE+cptEG+cptED+cptG+cptD)
    return depl

In [10]:
calculMouvements(logs=True,verbose =False)

{(0, -5): {'e': (0, -8),
  'eg': (4, -7),
  'ee': (0, -14),
  'eegg': (12, -7),
  'g': (3, -4),
  'gg': (4, -2)},
 (0, -8): {'a': (0, -5),
  'ag': (3, -4),
  'e': (0, -14),
  'eg': (7, -12),
  'ee': (0, -24),
  'eegg': (21, -12),
  'g': (4, -7),
  'gg': (7, -4)},
 (0, -14): {'a': (0, -8),
  'ag': (4, -7),
  'aa': (0, -5),
  'aagg': (4, -2),
  'e': (0, -24),
  'eg': (12, -21),
  'ee': (0, -41),
  'eegg': (36, -20),
  'g': (7, -12),
  'gg': (12, -7)},
 (0, -24): {'a': (0, -14),
  'ag': (7, -12),
  'aa': (0, -8),
  'aagg': (7, -4),
  'e': (0, -41),
  'eg': (21, -36),
  'g': (12, -21),
  'gg': (21, -12)},
 (0, -41): {'a': (0, -24),
  'ag': (12, -21),
  'aa': (0, -14),
  'aagg': (12, -7),
  'g': (21, -36),
  'gg': (36, -20)},
 (3, -4): {'e': (4, -7),
  'ed': (0, -8),
  'eg': (7, -4),
  'ee': (7, -12),
  'eegg': (14, 0),
  'd': (0, -5),
  'g': (4, -2),
  'gg': (5, 0)},
 (4, -7): {'a': (3, -4),
  'ad': (0, -5),
  'ag': (4, -2),
  'e': (7, -12),
  'ed': (0, -14),
  'eg': (12, -7),
  'ee': (12,

In [11]:
l = calculMouvements(logs=True,verbose =False)
cpt = 0
for i in l.keys():
    for j in l[i].keys():
        cpt+=1
        #print(str(i[0])+","+str(i[0]))
        #print(str(l[i][j][0])+","+str(l[i][j][1]))
print(cpt)

364


In [12]:
def creationListesMouvements(diagonales = False,doubleSaut = False,nombreAngle = 7, nombreDistance = 4, verbose = True,logs= False, dist = []):
    mvm = calculMouvements(nombreAngle = nombreAngle, nombreDistance = nombreDistance, verbose=verbose, logs=logs,dist = dist)
    a = []
    ag = []
    ad = []
    e = []
    eg = []
    ed = []
    g = []
    d = []
    listeMouvements = []
    if(verbose):
        print("Nombre de positions initiales")
        print(len(mvm.keys()))
    for i in mvm.keys():
        if("a" in list(mvm[i].keys())):
            a.append((i,mvm[i]["a"],"a"))
            if(doubleSaut and "aa" in list(mvm[i].keys())):
                a.append((i,mvm[i]["a"],"a"))
        if("ag" in list(mvm[i].keys())):
            ag.append((i,mvm[i]["ag"],"ag"))
            if(doubleSaut and "aagg" in list(mvm[i].keys())):
                ag.append((i,mvm[i]["ag"],"ag"))
        if("ad" in list(mvm[i].keys())):
            ad.append((i,mvm[i]["ad"],"ad"))
            if(doubleSaut and "aadd" in list(mvm[i].keys())):
                ad.append((i,mvm[i]["ad"],"ad"))
        if("e" in list(mvm[i].keys())):
            e.append((i,mvm[i]["e"],"e"))
            if(doubleSaut and "ee" in list(mvm[i].keys())):
                e.append((i,mvm[i]["e"],"e"))
        if("eg" in list(mvm[i].keys())):
            eg.append((i,mvm[i]["eg"],"eg"))
            if(doubleSaut and "eegg" in list(mvm[i].keys())):
                eg.append((i,mvm[i]["eg"],"eg"))
        if("ed" in list(mvm[i].keys())):
            ed.append((i,mvm[i]["ed"],"ed"))
            if(doubleSaut and "eedd" in list(mvm[i].keys())):
                ed.append((i,mvm[i]["ed"],"ed"))
        if("g" in list(mvm[i].keys())):
            g.append((i,mvm[i]["g"],"g"))
            if(doubleSaut and "gg" in list(mvm[i].keys())):
                g.append((i,mvm[i]["g"],"g"))
        if("d" in list(mvm[i].keys())):
            d.append((i,mvm[i]["d"],"d"))
            if(doubleSaut and "dd" in list(mvm[i].keys())):
                d.append((i,mvm[i]["d"],"d"))
    choices = []
    choices.append(a)
    choices.append(e)
    choices.append(g)
    choices.append(d)
    if(verbose and not diagonales):
        print("Nombre de choix de mouvements")
        print(len(choices[0])+len(choices[1])+len(choices[2])+len(choices[3]))
    if( diagonales):
        choices.append(ag)
        choices.append(ad)
        choices.append(eg)
        choices.append(ed)
        
    if(verbose and diagonales):
        print("Nombre de choix de mouvements avec diagonales")
        print(len(choices[0])+len(choices[1])+len(choices[2])+len(choices[3])+len(choices[4])+len(choices[5])+len(choices[6])+len(choices[7]))
    
    previousPos=(0,0)
    previousInd = -1
    while(max(len(l) for l in choices)>0):
        #Récupère les indices dans choices correspondant aux listes de mouvements étant les plus long
        ml = list(choices.index(l) for l in choices if len(l) !=0)
        ind = previousInd
        cptBreaker = 10
        while(ind == previousInd and cptBreaker>0):
            ind = random.choice(ml)
            cptBreaker -= 1
        if(cptBreaker < 1):
            print("Failed to choose a different movement. Ending")
            return 0
        previousInd=ind
        pos = previousPos
        cptBreaker = 10
        while(pos[0] == previousPos[0] and pos[1] == previousPos[1] and cptBreaker>0):
            cptBreaker -= 1
            position = random.choice(choices[ind])
            pos = position[0]
        if(cptBreaker < 1):
            print("Failed to choose a different start location. Ending")
            return 0
        previousPos=pos
        choices[ind].remove(position)
        listeMouvements.append(position)
    return listeMouvements

In [13]:
l = creationListesMouvements(diagonales = False,doubleSaut = False,nombreAngle = 7, nombreDistance = 4, verbose = True,logs= True, dist = dists)

Position intiales :(0, -5)
Position intiales :(0, -8)
Position intiales :(0, -14)
Position intiales :(0, -24)
Position intiales :(0, -41)
Position intiales :(3, -4)
Position intiales :(4, -7)
Position intiales :(7, -12)
Position intiales :(12, -21)
Position intiales :(21, -36)
Position intiales :(4, -2)
Position intiales :(7, -4)
Position intiales :(12, -7)
Position intiales :(21, -12)
Position intiales :(36, -20)
Position intiales :(5, 0)
Position intiales :(8, 0)
Position intiales :(14, 0)
Position intiales :(24, 0)
Position intiales :(41, 0)
Position intiales :(4, 2)
Position intiales :(7, 4)
Position intiales :(12, 7)
Position intiales :(21, 12)
Position intiales :(36, 20)
Position intiales :(3, 4)
Position intiales :(4, 7)
Position intiales :(7, 12)
Position intiales :(12, 21)
Position intiales :(21, 36)
Position intiales :(0, 5)
Position intiales :(0, 8)
Position intiales :(0, 14)
Position intiales :(0, 24)
Position intiales :(0, 41)
Nombre de pos initiales étudiées 35
Nombre de 

In [14]:
creationListesMouvements(diagonales = False,doubleSaut = False,nombreAngle = 7, nombreDistance = 4, verbose = False,logs= True, dist = dists)

[((4, 2), (3, 4), 'g'),
 ((0, 41), (0, 24), 'a'),
 ((41, 0), (36, -20), 'd'),
 ((8, 0), (5, 0), 'a'),
 ((7, 4), (12, 7), 'e'),
 ((4, -7), (0, -8), 'd'),
 ((7, -4), (12, -7), 'e'),
 ((21, -36), (0, -41), 'd'),
 ((0, -14), (0, -24), 'e'),
 ((4, -7), (3, -4), 'a'),
 ((4, 7), (0, 8), 'g'),
 ((21, 36), (12, 21), 'a'),
 ((14, 0), (12, 7), 'g'),
 ((24, 0), (41, 0), 'e'),
 ((0, 41), (21, 36), 'd'),
 ((36, -20), (41, 0), 'g'),
 ((4, 2), (5, 0), 'd'),
 ((4, 7), (7, 12), 'e'),
 ((0, 5), (3, 4), 'd'),
 ((7, -12), (12, -7), 'g'),
 ((21, -12), (36, -20), 'e'),
 ((8, 0), (7, -4), 'd'),
 ((4, -7), (7, -4), 'g'),
 ((12, -21), (0, -24), 'd'),
 ((0, 8), (0, 14), 'e'),
 ((41, 0), (36, 20), 'g'),
 ((12, 21), (7, 12), 'a'),
 ((7, 12), (0, 14), 'g'),
 ((14, 0), (8, 0), 'a'),
 ((0, 8), (4, 7), 'd'),
 ((8, 0), (14, 0), 'e'),
 ((0, 24), (0, 14), 'a'),
 ((3, -4), (0, -5), 'd'),
 ((0, -14), (7, -12), 'g'),
 ((4, -2), (7, -4), 'e'),
 ((3, -4), (4, -2), 'g'),
 ((0, 14), (7, 12), 'd'),
 ((7, -12), (4, -7), 'a'),
 ((

In [15]:
def posMvmToPresentationTemplate(p,diagonales = False):
    ind = -1
    if(not diagonales):
        if(p[2] == 'a'):
            ind = 2
        if(p[2] == 'e'):
            ind = 8
        if(p[2] == 'g'):
            ind = 4
        if(p[2] == 'd'):
            ind = 6
    else:
        if(p[2] == 'a'):
            ind = 2
        if(p[2] == 'e'):
            ind = 8
        if(p[2] == 'g'):
            ind = 4
        if(p[2] == 'd'):
            ind = 6
        if(p[2] == 'ag'):
            ind = 1
        if(p[2] == 'ad'):
            ind = 3
        if(p[2] == 'eg'):
            ind = 7
        if(p[2] == 'ed'):
            ind = 9
    pos00=""
    if(p[0][0]<0):
        pos00="m"+str(-p[0][0])
    else:
        pos00=str(p[0][0])
    pos01=str(p[0][1])
    if(p[0][1]<0):
        pos01="m"+str(-p[0][1])
    else:
        pos01=str(p[0][1])
    pos10=""
    if(p[1][0]<0):
        pos10="m"+str(-p[1][0])
    else:
        pos10=str(p[1][0])
    pos11=""
    if(p[1][1]<0):
        pos11="m"+str(-p[1][1])
    else:
        pos11=str(p[1][1])
    st = str(ind)+" sound"+pos00+"_"+pos01+" sound"+pos10+"_"+pos11+";"
    return st

In [16]:
def printAllMovement(verbose = False,diagonales = False,doubleSaut = False,arraySound = False, nombreAngle = 7, nombreDistance = 4,logs= False, dist = []):
    sounds = ['sound0_m5','sound3_m4','sound4_m2','sound5_0','sound4_2','sound3_4','sound0_5','sound0_m10','sound5_m9','sound9_m5','sound10_0','sound9_5','sound5_9', 'sound0_10','sound0_m15','sound8_m13','sound13_m7','sound15_0', 'sound13_7','sound8_13','sound0_15','sound0_m20','sound10_m17','sound17_m10','sound20_0','sound17_10','sound10_17','sound0_20']
    listeMvm = creationListesMouvements(verbose = verbose,diagonales = diagonales,doubleSaut=doubleSaut,nombreAngle = nombreAngle, nombreDistance = nombreDistance,logs=logs,dist=dist)
    if(listeMvm == 0):
        print("Error return 0")
        return 0
    cpt = 0
    for i in listeMvm:
        cpt+=1
        if(arraySound):
            p = posMvmToPresentationTemplate(i,diagonales = diagonales).split(" ")
        print(posMvmToPresentationTemplate(i,diagonales = diagonales))
    print("Nombre de mouvement effectuable : "+str(cpt))

In [17]:
l = calculPositionDisponibles(nombreAngle = 7,logs=True,dist=dists)
sounds = []
for i,j in l:
    s = "sound"
    if(i<0):
        s+="m"+str(-i)
    else:
        s+=str(i)
    s+="_"
    if(j<0):
        s+="m"+str(-j)
    else:
        s+=str(j)
    #print(s)
    sounds.append(s)
print("Nombre de positions disponibles : "+ str(len(sounds)))
sounds


Nombre de positions disponibles : 35


['sound0_m5',
 'sound3_m4',
 'sound4_m2',
 'sound5_0',
 'sound4_2',
 'sound3_4',
 'sound0_5',
 'sound0_m8',
 'sound4_m7',
 'sound7_m4',
 'sound8_0',
 'sound7_4',
 'sound4_7',
 'sound0_8',
 'sound0_m14',
 'sound7_m12',
 'sound12_m7',
 'sound14_0',
 'sound12_7',
 'sound7_12',
 'sound0_14',
 'sound0_m24',
 'sound12_m21',
 'sound21_m12',
 'sound24_0',
 'sound21_12',
 'sound12_21',
 'sound0_24',
 'sound0_m41',
 'sound21_m36',
 'sound36_m20',
 'sound41_0',
 'sound36_20',
 'sound21_36',
 'sound0_41']

In [18]:
#Liste des fichiers sons a mettre dans la partie définition des variables dans Presentation
sounds
for i in sounds:
    s=i.split("sound")[1]
    print("sound { wavefile { filename = \"1000ms/audioMosquito"+s+".wav\";};} sound"+s+";")

sound { wavefile { filename = "1000ms/audioMosquito0_m5.wav";};} sound0_m5;
sound { wavefile { filename = "1000ms/audioMosquito3_m4.wav";};} sound3_m4;
sound { wavefile { filename = "1000ms/audioMosquito4_m2.wav";};} sound4_m2;
sound { wavefile { filename = "1000ms/audioMosquito5_0.wav";};} sound5_0;
sound { wavefile { filename = "1000ms/audioMosquito4_2.wav";};} sound4_2;
sound { wavefile { filename = "1000ms/audioMosquito3_4.wav";};} sound3_4;
sound { wavefile { filename = "1000ms/audioMosquito0_5.wav";};} sound0_5;
sound { wavefile { filename = "1000ms/audioMosquito0_m8.wav";};} sound0_m8;
sound { wavefile { filename = "1000ms/audioMosquito4_m7.wav";};} sound4_m7;
sound { wavefile { filename = "1000ms/audioMosquito7_m4.wav";};} sound7_m4;
sound { wavefile { filename = "1000ms/audioMosquito8_0.wav";};} sound8_0;
sound { wavefile { filename = "1000ms/audioMosquito7_4.wav";};} sound7_4;
sound { wavefile { filename = "1000ms/audioMosquito4_7.wav";};} sound4_7;
sound { wavefile { filenam

In [19]:
listeMvm = 0
listeMvm2 = 0
while(listeMvm == 0):
    listeMvm = creationListesMouvements(verbose = False,diagonales = False,doubleSaut=False,logs=True,dist=dists)
while(listeMvm2 == 0):
    listeMvm2 = creationListesMouvements(verbose = False,diagonales = False,doubleSaut=False,logs=True,dist=dists)
same = True
for i in range(0,len(listeMvm)):
    if(listeMvm[i]!=listeMvm2[i]):
        same=False
        print(listeMvm)
        break
        
    listeMvm==listeMvm2

Failed to choose a different movement. Ending
Failed to choose a different start location. Ending
Failed to choose a different movement. Ending
Failed to choose a different movement. Ending
Failed to choose a different movement. Ending
[((21, -12), (24, 0), 'g'), ((36, -20), (21, -12), 'a'), ((5, 0), (8, 0), 'e'), ((0, 14), (7, 12), 'd'), ((12, -7), (21, -12), 'e'), ((12, -21), (21, -12), 'g'), ((0, -8), (0, -14), 'e'), ((4, 7), (3, 4), 'a'), ((3, 4), (4, 7), 'e'), ((4, -2), (3, -4), 'd'), ((0, -24), (12, -21), 'g'), ((0, 5), (3, 4), 'd'), ((21, -36), (36, -20), 'g'), ((4, -7), (7, -12), 'e'), ((0, -14), (0, -8), 'a'), ((4, 2), (7, 4), 'e'), ((3, -4), (0, -5), 'd'), ((36, -20), (41, 0), 'g'), ((21, 12), (12, 7), 'a'), ((0, -5), (0, -8), 'e'), ((4, 2), (5, 0), 'd'), ((21, 36), (12, 21), 'a'), ((12, 7), (21, 12), 'e'), ((12, 21), (21, 12), 'd'), ((7, -12), (12, -7), 'g'), ((21, -12), (12, -7), 'a'), ((14, 0), (12, -7), 'd'), ((4, -2), (5, 0), 'g'), ((0, 24), (0, 14), 'a'), ((12, 7), (14,

In [20]:
while(listeMvm == 0):
    listeMvm = creationListesMouvements(verbose = False,diagonales = False,doubleSaut=False,logs=True,dist=dists)

ar = "";
ids = "";
cpt=0
for i in listeMvm:
    cpt+=1
    print("pathSave.push_back({ "+str(i[0][0])+","+str(i[0][1])+","+str(i[1][0])+","+str(i[1][1])+" });")
print("pathSave.push_back({ 0,0,0,0 });")

pathSave.push_back({ 21,-12,24,0 });
pathSave.push_back({ 36,-20,21,-12 });
pathSave.push_back({ 5,0,8,0 });
pathSave.push_back({ 0,14,7,12 });
pathSave.push_back({ 12,-7,21,-12 });
pathSave.push_back({ 12,-21,21,-12 });
pathSave.push_back({ 0,-8,0,-14 });
pathSave.push_back({ 4,7,3,4 });
pathSave.push_back({ 3,4,4,7 });
pathSave.push_back({ 4,-2,3,-4 });
pathSave.push_back({ 0,-24,12,-21 });
pathSave.push_back({ 0,5,3,4 });
pathSave.push_back({ 21,-36,36,-20 });
pathSave.push_back({ 4,-7,7,-12 });
pathSave.push_back({ 0,-14,0,-8 });
pathSave.push_back({ 4,2,7,4 });
pathSave.push_back({ 3,-4,0,-5 });
pathSave.push_back({ 36,-20,41,0 });
pathSave.push_back({ 21,12,12,7 });
pathSave.push_back({ 0,-5,0,-8 });
pathSave.push_back({ 4,2,5,0 });
pathSave.push_back({ 21,36,12,21 });
pathSave.push_back({ 12,7,21,12 });
pathSave.push_back({ 12,21,21,12 });
pathSave.push_back({ 7,-12,12,-7 });
pathSave.push_back({ 21,-12,12,-7 });
pathSave.push_back({ 14,0,12,-7 });
pathSave.push_back({ 4,-2,5,0 

In [21]:
listeMvm

[((21, -12), (24, 0), 'g'),
 ((36, -20), (21, -12), 'a'),
 ((5, 0), (8, 0), 'e'),
 ((0, 14), (7, 12), 'd'),
 ((12, -7), (21, -12), 'e'),
 ((12, -21), (21, -12), 'g'),
 ((0, -8), (0, -14), 'e'),
 ((4, 7), (3, 4), 'a'),
 ((3, 4), (4, 7), 'e'),
 ((4, -2), (3, -4), 'd'),
 ((0, -24), (12, -21), 'g'),
 ((0, 5), (3, 4), 'd'),
 ((21, -36), (36, -20), 'g'),
 ((4, -7), (7, -12), 'e'),
 ((0, -14), (0, -8), 'a'),
 ((4, 2), (7, 4), 'e'),
 ((3, -4), (0, -5), 'd'),
 ((36, -20), (41, 0), 'g'),
 ((21, 12), (12, 7), 'a'),
 ((0, -5), (0, -8), 'e'),
 ((4, 2), (5, 0), 'd'),
 ((21, 36), (12, 21), 'a'),
 ((12, 7), (21, 12), 'e'),
 ((12, 21), (21, 12), 'd'),
 ((7, -12), (12, -7), 'g'),
 ((21, -12), (12, -7), 'a'),
 ((14, 0), (12, -7), 'd'),
 ((4, -2), (5, 0), 'g'),
 ((0, 24), (0, 14), 'a'),
 ((12, 7), (14, 0), 'd'),
 ((3, -4), (4, -2), 'g'),
 ((5, 0), (4, -2), 'd'),
 ((0, 41), (0, 24), 'a'),
 ((12, 21), (0, 24), 'g'),
 ((0, -41), (0, -24), 'a'),
 ((3, 4), (0, 5), 'g'),
 ((0, 8), (0, 14), 'e'),
 ((7, -4), (8, 

In [22]:
while(listeMvm == 0):
    listeMvm = creationListesMouvements(verbose = False,diagonales = False,doubleSaut=False,logs=True,dist=dists)

for i in listeMvm:
    print(i[2])
    print("sound { wavefile { filename = \"Continu/audioMosquitoFrom"+str(i[0][0])+"_"+str(i[0][1])+"to"+str(i[1][0])+"_"+str(i[1][1])+".wav\";};} soundFrom"+str(i[0][0])+"_"+str(i[0][1])+"to"+str(i[1][0])+"_"+str(i[1][1])+";")

g
sound { wavefile { filename = "Continu/audioMosquitoFrom21_-12to24_0.wav";};} soundFrom21_-12to24_0;
a
sound { wavefile { filename = "Continu/audioMosquitoFrom36_-20to21_-12.wav";};} soundFrom36_-20to21_-12;
e
sound { wavefile { filename = "Continu/audioMosquitoFrom5_0to8_0.wav";};} soundFrom5_0to8_0;
d
sound { wavefile { filename = "Continu/audioMosquitoFrom0_14to7_12.wav";};} soundFrom0_14to7_12;
e
sound { wavefile { filename = "Continu/audioMosquitoFrom12_-7to21_-12.wav";};} soundFrom12_-7to21_-12;
g
sound { wavefile { filename = "Continu/audioMosquitoFrom12_-21to21_-12.wav";};} soundFrom12_-21to21_-12;
e
sound { wavefile { filename = "Continu/audioMosquitoFrom0_-8to0_-14.wav";};} soundFrom0_-8to0_-14;
a
sound { wavefile { filename = "Continu/audioMosquitoFrom4_7to3_4.wav";};} soundFrom4_7to3_4;
e
sound { wavefile { filename = "Continu/audioMosquitoFrom3_4to4_7.wav";};} soundFrom3_4to4_7;
d
sound { wavefile { filename = "Continu/audioMosquitoFrom4_-2to3_-4.wav";};} soundFrom4_-2to

In [23]:
#Pour obtenir un nouvel arrangement de mouvements
#copier les deux affichages produits et remplacer les lignes correspondantes dans Presentation
listeMvm = 0
while(listeMvm == 0):
    listeMvm = creationListesMouvements(verbose = False,diagonales = False,doubleSaut=False,logs=True,dist=dists)
sounds = dict()
for i in range(0,len(listeMvm)):
    m = -1
    if(listeMvm[i][2] == "e"):
        m=8
    if(listeMvm[i][2] == "a"):
        m=2
    if(listeMvm[i][2] == "g"):
        m=4
    if(listeMvm[i][2] == "d"):
        m=6
    sounds[i] = (listeMvm[i][0],listeMvm[i][1],m)
    p00 = listeMvm[i][0][0]
    p01 = listeMvm[i][0][1]
    p10 = listeMvm[i][1][0]
    p11 = listeMvm[i][1][1]
    st = "sound { wavefile { filename = \"Continu/audioMosquitoFrom"+str(p00)+"_"+str(p01)+"to"+str(p10)+"_"+str(p11)+".wav\";};} soundFrom"
    if(p00<0):
        p00="m"+str(-p00)
    else:
        p00=str(p00)
    if(p01<0):
        p01="m"+str(-p01)
    else:
        p01=str(p01)
    if(p10<0):
        p10="m"+str(-p10)
    else:
        p10=str(p10)
    if(p11<0):
        p11="m"+str(-p11)
    else:
        p11=str(p11)
    st+=p00+"_"+p01+"to"+p10+"_"+p11+";"
    print(st)
ar = "";
ids = "";
for ind in sounds.keys():
    p1 = sounds[ind][0]
    p2 = sounds[ind][1]
    m = sounds[ind][2]
    ar+="{"+str(p1[0])+","+str(p1[1])+","+str(p2[0])+","+str(p2[1])+"},"
    ids+=str(m)+","
ar ="array <int>  positions["+str(len(listeMvm))+"][2] = {"+ar[:-1]+"};"
ids="array <int> idsMvm["+str(len(listeMvm))+"] = {"+ids[:-1]+"};"
print(ar)
print(ids)

Failed to choose a different movement. Ending
Failed to choose a different movement. Ending
Failed to choose a different movement. Ending
Failed to choose a different movement. Ending
Failed to choose a different movement. Ending
Failed to choose a different movement. Ending
Failed to choose a different movement. Ending
Failed to choose a different movement. Ending
Failed to choose a different movement. Ending
Failed to choose a different movement. Ending
Failed to choose a different movement. Ending
sound { wavefile { filename = "Continu/audioMosquitoFrom0_14to7_12.wav";};} soundFrom0_14to7_12;
sound { wavefile { filename = "Continu/audioMosquitoFrom12_21to0_24.wav";};} soundFrom12_21to0_24;
sound { wavefile { filename = "Continu/audioMosquitoFrom24_0to21_-12.wav";};} soundFrom24_0to21_m12;
sound { wavefile { filename = "Continu/audioMosquitoFrom4_-7to3_-4.wav";};} soundFrom4_m7to3_m4;
sound { wavefile { filename = "Continu/audioMosquitoFrom14_0to12_7.wav";};} soundFrom14_0to12_7;
sou

In [24]:
def produceMovements(diagonales,doubleSaut,nombreAngle = 7, nombreDistance = 4,verbose = False,logs=False,dist=[]):
    x = 0
    while(x == 0):
        x = printAllMovement(diagonales = diagonales,verbose = verbose,nombreAngle = nombreAngle, nombreDistance = nombreDistance,doubleSaut=doubleSaut,logs=logs,dist=dist)

In [25]:
produceMovements(diagonales = False,doubleSaut = False, nombreAngle = 7,logs=True,dist=dists)

6 sound36_20 sound41_0;
4 sound12_7 sound7_12;
2 sound24_0 sound14_0;
8 sound4_7 sound7_12;
4 sound12_m7 sound14_0;
6 sound0_24 sound12_21;
8 sound12_m21 sound21_m36;
2 sound21_36 sound12_21;
4 sound36_20 sound21_36;
8 sound4_m7 sound7_m12;
6 sound7_12 sound12_7;
4 sound21_m36 sound36_m20;
8 sound7_12 sound12_21;
4 sound4_2 sound3_4;
2 sound14_0 sound8_0;
4 sound0_m24 sound12_m21;
8 sound3_m4 sound4_m7;
4 sound0_m41 sound21_m36;
6 sound12_m7 sound7_m12;
8 sound14_0 sound24_0;
2 sound7_12 sound4_7;
8 sound0_8 sound0_14;
6 sound8_0 sound7_m4;
8 sound21_m12 sound36_m20;
6 sound5_0 sound4_m2;
4 sound4_m7 sound7_m4;
2 sound36_m20 sound21_m12;
4 sound0_m14 sound7_m12;
8 sound7_m12 sound12_m21;
6 sound4_m2 sound3_m4;
2 sound12_m21 sound7_m12;
6 sound21_36 sound36_20;
4 sound7_m12 sound12_m7;
2 sound12_21 sound7_12;
8 sound7_4 sound12_7;
2 sound0_m41 sound0_m24;
6 sound21_m12 sound12_m21;
8 sound7_m4 sound12_m7;
4 sound12_m21 sound21_m12;
8 sound0_14 sound0_24;
6 sound7_m4 sound4_m7;
8 sound12